In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import json
## For plotting 
import matplotlib.pyplot as plt
## For stationarity test and decomposition
import statsmodels.tsa.api as smt
import statsmodels.api as sm

In [3]:
thead = pd.read_csv("../input/ubiquant-market-prediction/train.csv", nrows=5) # just read in a few lines to get the column headers
dtypes = dict(zip(thead.columns.values, ['object', 'int16', 'int16', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32', 'float32']))
del thead
print('Datatype used for each column:\n', json.dumps(dtypes))
df = pd.read_csv("../input/ubiquant-market-prediction/train.csv", dtype=dtypes, nrows=100000)
df.head()

In [4]:
#type individual time stamps
column_values = df[["time_id"]].values
unique_values =  np.unique(column_values)
print(unique_values)

In [6]:
def add_shift_time_col(df):
    #Sort by time_id 
    #print(df)
    #print(type(df))
    time_sorted_df = df
    time_sorted_df = time_sorted_df.sort_values(by ='time_id')
    #Shift col by one val 
    time_sorted_df['target_shift'] = time_sorted_df['target']
    #print('Vor shift', time_sorted_df['target_shift'])
    time_sorted_df['target_shift'] = time_sorted_df.target.shift(1)
    #print('Nach shift', time_sorted_df['target_shift'])
    #Aufpassen vllt macht der stress wegen nan wert in erster zelle
    mean_target_by_asset = time_sorted_df['target_shift'].mean()
    time_sorted_df['target_shift'].values[0] = mean_target_by_asset
    #print('Shift hat wert gefüllt', time_sorted_df['target_shift'])
    return time_sorted_df

In [7]:
#In jeder asset klasse korrelation der zeitversetzten returns berechnen, und für die asset klassen mit sehr geringer correlation 
#nehmen wir einen random walk und für die anderen deterministische Modelle

#1 Für jede Asset klasse eine zeitlich geordnete df machen
#1.1 shift funktion bauen 
#2 dann mit für jedes asset df time shift machen auf den returns
#3 korrelation berechnen für die returns aller asset klassen
#4.1 asset klassen mit keiner korrelation auf den returns -> random walk
#4.2 asset klassen mit einer 'hohen' korrelation lasso laufen lassen, dann random forest oder lin

#split by asset class

#erstmal asset class analytics -> siehe oben
#print(len(df[["investment_id"]]))
invest_id_col_vals = df[["investment_id"]].values
unique_invest_ids =  np.unique(invest_id_col_vals)
#print(unique_invest_ids)
#wir erschaffen ein df dictionary nach allen asset klassen 1...3772
dict_by_asset_class_shift = {elem : pd.DataFrame for elem in unique_invest_ids}
for key in dict_by_asset_class_shift.keys():
    dict_by_asset_class_shift[key] = df[:][df.investment_id == key]
    #print('Sollte nach invest_id geordnet', dict_by_asset_class_shift[key])
    dict_by_asset_class_shift[key] = add_shift_time_col(dict_by_asset_class_shift[key])
    #print('Sollte nach time geordnet', dict_by_asset_class_shift[key])

In [8]:
low_cor_key = []
high_cor_key = []

for key in dict_by_asset_class_shift:
    corrrelation_returns = np.corrcoef(dict_by_asset_class_shift[key]['target_shift'], dict_by_asset_class_shift[key]['target'])[0][1]
    dict_by_asset_class_shift[key].drop(['target_shift'], axis = 1)
    if corrrelation_returns < 0.35:
        low_cor_key.append(key)
    else:
        high_cor_key.append(key)

In [17]:
from collections import Counter
#print(len(low_cor_key))
high_cor_count_list = []
for key in high_cor_key:
    #print(len(dict_by_asset_class_shift[key]))
    high_cor_count_list.append(len(dict_by_asset_class_shift[key]))
high_cor_count_list_array = np.array(high_cor_count_list)
counter_high_cor = Counter(high_cor_count_list_array)

#plot histogram for number of observations in asset class (x-axis) and the frequency particular observation numbers occur (y-axis)
import matplotlib.pyplot as plt
plt.bar(counter_high_cor.keys(), counter_high_cor.values())

In [16]:
#Wie gehe ich damit um, dass es investment_ids gibt, die nur eine row enthalten und ids die mehrere rows enthalten
#ich muss die rausnehmen, die kaum werte haben und hoch korrelieren
#wir schauen uns an wie die frequency dist 
from collections import Counter
#print(len(low_cor_key))
low_cor_count_list = []
for key in low_cor_key:
    #print(len(dict_by_asset_class_shift[key]))
    low_cor_count_list.append(len(dict_by_asset_class_shift[key]))
low_cor_count_list_array = np.array(low_cor_count_list)
counter_low_cor = Counter(low_cor_count_list_array)

#plot histogram for number of observations in asset class (x-axis) and the frequency particular observation numbers occur (y-axis)
import matplotlib.pyplot as plt
plt.bar(counter_low_cor.keys(), counter_low_cor.values())

In [ ]:
dict_by_asset_class_shift[10]['pred_target']

In [ ]:
#Highly correlated
#load data
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
# define model
print(len(high_cor_key))
for key in high_cor_key:
    print(len(dict_by_asset_class_shift[key]))
print(len(high_cor_key))
model = Lasso(alpha=1.0)
# evaluate an lasso regression model on the dataset
for key in high_cor_key:
    print(dict_by_asset_class_shift[key] )
    if(len(dict_by_asset_class_shift[key]) < 10):
        continue
    model = Lasso(alpha=1.0)
    dataframe = dict_by_asset_class_shift[key] 
    data = dataframe.values
    X = dict_by_asset_class_shift[key].loc[:,"f_0":"f_299"]
    y = dict_by_asset_class_shift[key]['target']
    # define model
    model = Lasso(alpha=1.0)
    # define model evaluation method
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    # force scores to be positive
    scores = absolute(scores)
    print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    dtest = test_df.iloc[:,2:] #ADDED: remove row_id and investment_id
    pred = model.predict(dtest) #ADDED
    sample_prediction_df['target'] = pred  # CHANGED: replace 0 with pred
    env.predict(sample_prediction_df)   # register your predictions

In [ ]:
print(test_df)

In [ ]:
print(sample_prediction_df)